In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Stroke"
cohort = "GSE125771"

# Input paths
in_trait_dir = "../../input/GEO/Stroke"
in_cohort_dir = "../../input/GEO/Stroke/GSE125771"

# Output paths
out_data_file = "../../output/preprocess/Stroke/GSE125771.csv"
out_gene_data_file = "../../output/preprocess/Stroke/gene_data/GSE125771.csv"
out_clinical_data_file = "../../output/preprocess/Stroke/clinical_data/GSE125771.csv"
json_path = "../../output/preprocess/Stroke/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"RNA expression data from calcified human carotid atherosclerotic plaques"
!Series_summary	"Although unstable atherosclerosis in the carotid bifurcation is a significant etiology behind ischemic stroke, clinical imaging methods to distinguish stable from vulnerable lesions are lacking and selection of patients for stroke-preventive intervention still relies on surrogate variables with moderate predictive power, such as the degree of luminal narrowing. Here we combined clinical and diagnostic imaging information by comuted tomography to select patients with calcified plaques for large scale molecular analysis, in an effort to increase our understanding of the pathophysiology behind carotid plaque instability as related to patient- and plaque- phenotype."
!Series_overall_design	"Patients undergoing surgery for high-grade (>50% NASCET) carotid stenosis at the Department of Vascular Surgery, Karolinska University Hospital, Stockholm, Sweden were consec

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains RNA expression data from calcified human carotid atherosclerotic plaques
# It is likely to contain gene expression data as it mentions "RNA expression data" and "microarrays"
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (Stroke):
# This is a study on atherosclerotic plaques in carotid arteries, which are associated with stroke
# All samples are from carotid plaques, and the study mentions relation to ischemic stroke
# However, there's no explicit indication in the sample characteristics about which patients had stroke
# The study only mentions patients with high-grade carotid stenosis, not specifically stroke events
trait_row = None  # No explicit stroke data available in the sample characteristics

# For age:
# Age data is available at row 3
age_row = 3

# For gender:
# Gender data is available at row 2
gender_row = 2

# 2.2 Data Type Conversion

# Convert trait function (not used since trait_row is None)
def convert_trait(value):
    if value is None:
        return None
    value = value.split(': ')[-1].lower().strip()
    if value in ['yes', 'stroke', 'positive', '1']:
        return 1
    elif value in ['no', 'control', 'negative', '0']:
        return 0
    return None

# Convert age function
def convert_age(value):
    if value is None:
        return None
    try:
        # Extract the numeric value after the colon
        age_value = value.split(': ')[-1].strip()
        return float(age_value)  # Convert to float for continuous variable
    except (ValueError, AttributeError):
        return None

# Convert gender function
def convert_gender(value):
    if value is None:
        return None
    value = value.split(': ')[-1].lower().strip()
    if value in ['male', 'm']:
        return 1
    elif value in ['female', 'f']:
        return 0
    return None

# 3. Save Metadata
# Trait data is not available
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None (trait data is not available), we should skip this substep entirely
# per the instructions: "If trait_row is not None, it means clinical data is available, then you MUST DO this substep. 
# Otherwise, you should skip this substep."


A new JSON file was created at: ../../output/preprocess/Stroke/cohort_info.json


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Stroke/GSE125771/GSE125771_series_matrix.txt.gz
Gene data shape: (65535, 40)
First 20 gene/probe identifiers:
Index(['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1',
       'TC01000004.hg.1', 'TC01000005.hg.1', 'TC01000006.hg.1',
       'TC01000007.hg.1', 'TC01000008.hg.1', 'TC01000009.hg.1',
       'TC01000010.hg.1', 'TC01000011.hg.1', 'TC01000012.hg.1',
       'TC01000013.hg.1', 'TC01000014.hg.1', 'TC01000015.hg.1',
       'TC01000016.hg.1', 'TC01000017.hg.1', 'TC01000018.hg.1',
       'TC01000019.hg.1', 'TC01000020.hg.1'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers appear to be microarray probe IDs, not standard human gene symbols
# They follow the format "TC01000001.hg.1" which is consistent with the Affymetrix Transcriptome Array probe IDs
# These need to be mapped to human gene symbols for proper analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Let's look for platform information in the SOFT file to understand the annotation better
print("\nSearching for platform information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    for i, line in enumerate(f):
        if '!Series_platform_id' in line:
            print(line.strip())
            break
        if i > 100:  # Limit search to first 100 lines
            print("Platform ID not found in first 100 lines")
            break

# Check if the SOFT file includes any reference to gene symbols
print("\nSearching for gene symbol information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    gene_symbol_lines = []
    for i, line in enumerate(f):
        if 'GENE_SYMBOL' in line or 'gene_symbol' in line.lower() or 'symbol' in line.lower():
            gene_symbol_lines.append(line.strip())
        if i > 1000 and len(gene_symbol_lines) > 0:  # Limit search but ensure we found something
            break
    
    if gene_symbol_lines:
        print("Found references to gene symbols:")
        for line in gene_symbol_lines[:5]:  # Show just first 5 matches
            print(line)
    else:
        print("No explicit gene symbol references found in first 1000 lines")

# Look for alternative annotation files or references in the directory
print("\nChecking for additional annotation files in the directory:")
all_files = os.listdir(in_cohort_dir)
print([f for f in all_files if 'annotation' in f.lower() or 'platform' in f.lower() or 'gpl' in f.lower()])



Gene annotation preview:
Columns in gene annotation: ['ID', 'probeset_id', 'seqname', 'strand', 'start', 'stop', 'total_probes', 'gene_assignment', 'mrna_assignment', 'swissprot', 'unigene', 'category', 'locus type', 'notes', 'SPOT_ID']
{'ID': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'probeset_id': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['11869', '29554', '69091', '160446', '317811'], 'stop': ['14409', '31109', '70008', '161525', '328581'], 'total_probes': [49.0, 60.0, 30.0, 30.0, 191.0], 'gene_assignment': ['NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// ENST00000456328 // DDX11L5 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 5 // 9p24.3 // 100287596 /// ENST00000456328 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His)

No explicit gene symbol references found in first 1000 lines

Checking for additional annotation files in the directory:
[]


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Observe gene identifiers and determine mapping columns
# Based on the gene identifiers, we see that 'ID' column in the gene_annotation matches the gene expression data index
# The 'gene_assignment' column contains the gene symbols information

# 2. Get gene mapping dataframe by extracting relevant columns
# The probe identifiers are in the 'ID' column
# The gene symbols are in the 'gene_assignment' column
mapping_df = get_gene_mapping(gene_annotation, 'ID', 'gene_assignment')

print(f"Gene mapping shape: {mapping_df.shape}")
print("Sample of gene mapping data:")
print(mapping_df.head())

# 3. Convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Normalize gene symbols
gene_data = normalize_gene_symbols_in_index(gene_data) 

print(f"After mapping, gene data shape: {gene_data.shape}")
print("First 10 gene symbols after mapping:")
print(gene_data.index[:10])

# Save the processed gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene data saved to: {out_gene_data_file}")


Gene mapping shape: (70753, 2)
Sample of gene mapping data:
                ID                                               Gene
0  TC01000001.hg.1  NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-As...
1  TC01000002.hg.1  ENST00000408384 // MIR1302-11 // microRNA 1302...
2  TC01000003.hg.1  NM_001005484 // OR4F5 // olfactory receptor, f...
3  TC01000004.hg.1  OTTHUMT00000007169 // OTTHUMG00000002525 // NU...
4  TC01000005.hg.1  NR_028322 // LOC100132287 // uncharacterized L...


After mapping, gene data shape: (24012, 40)
First 10 gene symbols after mapping:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2ML1-AS1',
       'A2ML1-AS2', 'A2MP1', 'A4GALT'],
      dtype='object', name='Gene')


Gene data saved to: ../../output/preprocess/Stroke/gene_data/GSE125771.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols - this was already done in Step 6
print(f"Gene data shape already normalized: {gene_data.shape}")

# 2. Explore the clinical data structure to identify relevant rows
print("\nExploring clinical data structure:")
for i in range(min(10, len(clinical_data))):  # Check up to first 10 rows
    print(f"Row {i}:", list(clinical_data.iloc[i].unique())[:5])  # Show first 5 unique values

# Extract the sample characteristics dictionary again to analyze content
sample_characteristics_dict = get_unique_values_by_row(clinical_data)
print("\nSample characteristics dictionary:")
print(sample_characteristics_dict)

# Based on the exploration, we can see that clinical data doesn't have stroke information
# All samples are from carotid atherosclerotic plaques but without explicit stroke status
# We'll have to mark this dataset as not having the trait data available

# Save the gene expression data (already normalized in Step 6)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")

# Mark dataset as not having trait data available
is_trait_available = False
is_gene_available = True

# Initial validation with is_final=False since we don't have trait data
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

print("Dataset processing complete. The dataset has gene expression data but lacks stroke trait information.")

Gene data shape already normalized: (24012, 40)

Exploring clinical data structure:
Row 0: ['!Sample_characteristics_ch1', 'tissue: carotid-atherosclerotic-plaque']
Row 1: ['!Sample_characteristics_ch1', 'ID: sample1', 'ID: sample2', 'ID: sample3', 'ID: sample4']
Row 2: ['!Sample_characteristics_ch1', 'Sex: Male', 'Sex: Female']
Row 3: ['!Sample_characteristics_ch1', 'age: 73', 'age: 60', 'age: 81', 'age: 85']

Sample characteristics dictionary:
{0: ['tissue: carotid-atherosclerotic-plaque'], 1: ['ID: sample1', 'ID: sample2', 'ID: sample3', 'ID: sample4', 'ID: sample5', 'ID: sample6', 'ID: sample7', 'ID: sample8', 'ID: sample9', 'ID: sample10', 'ID: sample11', 'ID: sample12', 'ID: sample13', 'ID: sample14', 'ID: sample15', 'ID: sample16', 'ID: sample17', 'ID: sample18', 'ID: sample19', 'ID: sample20', 'ID: sample21', 'ID: sample22', 'ID: sample23', 'ID: sample24', 'ID: sample25', 'ID: sample26', 'ID: sample27', 'ID: sample28', 'ID: sample29', 'ID: sample30'], 2: ['Sex: Male', 'Sex: Fem

Gene expression data saved to ../../output/preprocess/Stroke/gene_data/GSE125771.csv
Dataset processing complete. The dataset has gene expression data but lacks stroke trait information.
